In [ ]:
!nvidia-smi


In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
    return DefaultPredictor(cfg)

def filter_predictions(outputs, class_id):
    # Filter out instances that are not the desired class
    instances = outputs["instances"]
    mask = instances.pred_classes == class_id
    instances = instances[mask]
    return instances

def find_outer_contour(mask):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def find_convex_hull(contour):
    # Find convex hull of the contour
    hull = cv2.convexHull(contour)
    return hull

def fit_lines_from_convex_hull(hull, num_lines):
    # Approximate lines from convex hull
    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, closed=True)
    
    # Fit lines through the approximated polygon
    lines = []
    if len(approx) >= 2:
        for i in range(len(approx) - 1):
            lines.append([approx[i][0], approx[i+1][0]])
        lines.append([approx[-1][0], approx[0][0]])
    
    return lines[:num_lines]

def extend_lines(lines, extension_length):
    # Extend each line segment in both directions
    extended_lines = []
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        
        # Calculate line direction
        dx = x2 - x1
        dy = y2 - y1
        
        # Extend in both directions
        extended_line1 = [(int(x1 - extension_length * dx), int(y1 - extension_length * dy)),
                          (int(x2 + extension_length * dx), int(y2 + extension_length * dy))]
        
        extended_lines.append(extended_line1)
    
    return extended_lines

def draw_lines(image, lines):
    # Draw lines on the image
    for line in lines:
        cv2.line(image, tuple(line[0]), tuple(line[1]), (0, 255, 0), 2)
    return image

def visualize_masks_with_extended_lines(image, instances, num_lines=5, extension_length=20):
    # Visualize masks and draw lines based on convex hull
    masks = instances.pred_masks.cpu().numpy()
    lines = []
    
    for mask in masks:
        contours = find_outer_contour(mask)
        for contour in contours:
            hull = find_convex_hull(contour)
            lines.extend(fit_lines_from_convex_hull(hull, num_lines))
    
    extended_lines = extend_lines(lines, extension_length)
    result_image = draw_lines(image.copy(), extended_lines)
    
    return result_image

def process_image(predictor, image_path, class_id):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Filter predictions to only include the desired class
    filtered_instances = filter_predictions(outputs, class_id)
    
    # Visualize masks with extended lines
    result_image = visualize_masks_with_extended_lines(image, filtered_instances)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path, class_id):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        
        # Filter predictions to only include the desired class
        filtered_instances = filter_predictions(outputs, class_id)
        
        # Visualize masks with extended lines
        result_frame = visualize_masks_with_extended_lines(frame, filtered_instances)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path, class_id):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, class_id)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, class_id)

# Example usage
config_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/config.yaml"
weights_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/model_final.pth"
input_folder = "D:/Tank_proj/tank_edge/tank_edges-1/test"

predictor = load_model(config_file, weights_file)
class_id = 2  # Assuming "object 2" corresponds to class ID 2
process_folder(predictor, input_folder, class_id)


In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
    return DefaultPredictor(cfg)

def filter_predictions(outputs, class_id):
    # Filter out instances that are not the desired class
    instances = outputs["instances"]
    mask = instances.pred_classes == class_id
    instances = instances[mask]
    return instances

def find_outer_contour(mask):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def find_convex_hull(contour):
    # Find convex hull of the contour
    hull = cv2.convexHull(contour)
    return hull

def fit_lines_from_convex_hull(hull, num_lines):
    # Approximate lines from convex hull
    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, closed=True)
    
    # Fit lines through the approximated polygon
    lines = []
    if len(approx) >= 2:
        for i in range(len(approx) - 1):
            lines.append([approx[i][0], approx[i+1][0]])
        lines.append([approx[-1][0], approx[0][0]])
    
    return lines[:num_lines]

def calculate_angle(x1, y1, x2, y2):
    # Calculate the angle of the line
    angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    return angle

def extend_lines(lines, extension_length, min_angle=0, max_angle=180):
    # Extend each line segment in both directions
    extended_lines = []
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        
        # Calculate line direction
        dx = x2 - x1
        dy = y2 - y1
        
        # Calculate angle
        angle = calculate_angle(x1, y1, x2, y2)
        
        # Check if angle is within the specified range
        if min_angle <= angle <= max_angle:
            # Extend in both directions
            extended_line1 = [(int(x1 - extension_length * dx), int(y1 - extension_length * dy)),
                              (int(x2 + extension_length * dx), int(y2 + extension_length * dy))]
            extended_lines.append((extended_line1, angle))
    
    return extended_lines

def draw_lines(image, lines):
    # Draw lines and angles on the image
    for line, angle in lines:
        cv2.line(image, tuple(line[0]), tuple(line[1]), (0, 255, 0), 2)
        # Calculate midpoint of the line
        mid_x = (line[0][0] + line[1][0]) // 2
        mid_y = (line[0][1] + line[1][1]) // 2
        # Draw the angle at the midpoint
        cv2.putText(image, f"{angle:.1f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    return image

def visualize_masks_with_extended_lines(image, instances, num_lines=5, extension_length=20, min_angle=0, max_angle=180):
    # Visualize masks and draw lines based on convex hull
    masks = instances.pred_masks.cpu().numpy()
    lines = []
    
    for mask in masks:
        contours = find_outer_contour(mask)
        for contour in contours:
            hull = find_convex_hull(contour)
            lines.extend(fit_lines_from_convex_hull(hull, num_lines))
    
    extended_lines = extend_lines(lines, extension_length, min_angle, max_angle)
    result_image = draw_lines(image.copy(), extended_lines)
    
    return result_image

def process_image(predictor, image_path, class_id, min_angle=0, max_angle=180):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Filter predictions to only include the desired class
    filtered_instances = filter_predictions(outputs, class_id)
    
    # Visualize masks with extended lines
    result_image = visualize_masks_with_extended_lines(image, filtered_instances, min_angle=min_angle, max_angle=max_angle)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path, class_id, min_angle=0, max_angle=180):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        
        # Filter predictions to only include the desired class
        filtered_instances = filter_predictions(outputs, class_id)
        
        # Visualize masks with extended lines
        result_frame = visualize_masks_with_extended_lines(frame, filtered_instances, min_angle=min_angle, max_angle=max_angle)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path, class_id, min_angle=0, max_angle=180):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, class_id, min_angle=min_angle, max_angle=max_angle)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, class_id, min_angle=min_angle, max_angle=max_angle)

# Example usage
config_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/config.yaml"
weights_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/model_final.pth"
input_folder = "D:/Tank_proj/tank_edge/tank_edges-1/test"

predictor = load_model(config_file, weights_file)
class_id = 2  # Assuming "object 2" corresponds to class ID 2
min_angle = 80  # Example minimum angle in degrees
max_angle = 130  # Example maximum angle in degrees
process_folder(predictor, input_folder, class_id, min_angle, max_angle)


In [8]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    # Load the configuration
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
    return DefaultPredictor(cfg)

def filter_predictions(outputs, class_id):
    # Filter out instances that are not the desired class
    instances = outputs["instances"]
    mask = instances.pred_classes == class_id
    instances = instances[mask]
    return instances

def find_outer_contour(mask):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def find_convex_hull(contour):
    # Find convex hull of the contour
    hull = cv2.convexHull(contour)
    return hull

def fit_lines_from_convex_hull(hull, num_lines):
    # Approximate lines from convex hull
    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, closed=True)
    
    # Fit lines through the approximated polygon
    lines = []
    if len(approx) >= 2:
        for i in range(len(approx) - 1):
            lines.append([approx[i][0], approx[i+1][0]])
        lines.append([approx[-1][0], approx[0][0]])
    
    return lines[:num_lines]

def calculate_angle(x1, y1, x2, y2):
    # Calculate the angle of the line
    angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    return angle

def extend_lines(lines, extension_length, min_angle=0, max_angle=180):
    # Extend each line segment in both directions and find the line with the largest angle
    largest_angle = -1
    largest_angle_line = None
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        
        # Calculate line direction
        dx = x2 - x1
        dy = y2 - y1
        
        # Calculate angle
        angle = calculate_angle(x1, y1, x2, y2)
        
        # Check if angle is within the specified range
        if min_angle <= angle <= max_angle and angle > largest_angle:
            largest_angle = angle
            largest_angle_line = [(int(x1 - extension_length * dx), int(y1 - extension_length * dy)),
                                  (int(x2 + extension_length * dx), int(y2 + extension_length * dy))]
    
    return [(largest_angle_line, largest_angle)] if largest_angle_line else []

def draw_lines(image, lines):
    # Draw lines and angles on the image
    for line, angle in lines:
        cv2.line(image, tuple(line[0]), tuple(line[1]), (0, 255, 0), 2)
        # Calculate midpoint of the line
        mid_x = (line[0][0] + line[1][0]) // 2
        mid_y = (line[0][1] + line[1][1]) // 2
        # Draw the angle at the midpoint
        cv2.putText(image, f"{angle:.1f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    return image

def visualize_masks_with_extended_lines(image, instances, num_lines=5, extension_length=20, min_angle=0, max_angle=180):
    # Visualize masks and draw lines based on convex hull
    masks = instances.pred_masks.cpu().numpy()
    lines = []
    
    for mask in masks:
        contours = find_outer_contour(mask)
        for contour in contours:
            hull = find_convex_hull(contour)
            lines.extend(fit_lines_from_convex_hull(hull, num_lines))
    
    extended_lines = extend_lines(lines, extension_length, min_angle, max_angle)
    result_image = draw_lines(image.copy(), extended_lines)
    
    return result_image

def process_image(predictor, image_path, class_id, min_angle=0, max_angle=180):
    # Load image
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    # Filter predictions to only include the desired class
    filtered_instances = filter_predictions(outputs, class_id)
    
    # Visualize masks with extended lines
    result_image = visualize_masks_with_extended_lines(image, filtered_instances, min_angle=min_angle, max_angle=max_angle)
    
    # Display the image
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path, class_id, min_angle=0, max_angle=180):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        outputs = predictor(frame)
        
        # Filter predictions to only include the desired class
        filtered_instances = filter_predictions(outputs, class_id)
        
        # Visualize masks with extended lines
        result_frame = visualize_masks_with_extended_lines(frame, filtered_instances, min_angle=min_angle, max_angle=max_angle)
        
        # Display the frame
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path, class_id, min_angle=0, max_angle=180):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, class_id, min_angle=min_angle, max_angle=max_angle)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, class_id, min_angle=min_angle, max_angle=max_angle)

# Example usage
config_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/config.yaml"
weights_file = "D:/Tank_proj/tank_edge/tank-edge/mask_rcnn_R_101_FPN_3x/2024-06-29-23-12-58/model_final.pth"
input_folder = "D:/Tank_proj/tank_edge/tank_edges-1/test"

predictor = load_model(config_file, weights_file)
class_id = 2  # Assuming "object 2" corresponds to class ID 2
min_angle = 45  # Example minimum angle in degrees
max_angle = 135  # Example maximum angle in degrees
process_folder(predictor, input_folder, class_id, min_angle, max_angle)
